In [1]:
import livef1

session = livef1.get_session(
    season = 2025,
    meeting_identifier = "Shakir",
    session_identifier = "Practice 3"
)

session.generate()

# session = livef1.get_session(
#     season=2024,
#     meeting_identifier="Spa",
#     session_identifier="Practice 2"
# )

# session.generate()

01:29:24 - The identifier couldn't be found.
01:29:24 - The identifier is very close to 'Sakhir' at column 'MEETING CIRCUIT SHORTNAME'
01:29:24 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 ARAMCO PRE-SEASON TESTING 2025
	Meeting Name : Pre-Season Testing
	Meeting Circuit Shortname : Sakhir
01:29:24 - Got the meeting.
01:29:24 - Selected meeting/session is:
	Session Name : Day 3
01:29:24 - The session was received successfully.
01:29:24 - Fetching drivers.
01:29:24 - Fetching data : 'CarData.z'
01:29:24 - Fetching data : 'SessionStatus'
01:29:24 - Fetching data : 'Position.z'
01:29:24 - Fetching data : 'TimingData'
01:29:24 - 'SessionStatus' has been fetched and parsed
01:29:25 - 'TimingData' has been fetched and parsed
01:29:30 - 'Position.z' has been fetched and parsed
01:29:33 - 'CarData.z' has been fetched and parsed
01:29:45 - Silver tables are being generated.


_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']


01:29:49 - 'laps' has been generated and saved to the silver lake. You can access it from 'session.laps'.


_deleted ['NumberOfLaps', 'NumberOfPitStops']
_deleted ['Lap']
_deleted ['Lap']


01:30:06 - 'carTelemetry' has been generated and saved to the silver lake. You can access it from 'session.carTelemetry'.


In [ ]:
session.laps

In [2]:
df_test = session.laps.copy()
df_test["tot"] = df_test["sector1_time"] + df_test["sector2_time"] + df_test["sector3_time"]
df_test[df_test["DriverNo"] == "18"].sort_values("sector2_time",ascending=True)

,lap_number,lap_time,in_pit,pit_out,sector1_time,sector2_time,sector3_time,None,speed_I1,speed_I2,speed_FL,speed_ST,no_pits,lap_start_time,DriverNo,lap_start_date,tot
1349,22,0 days 00:01:31.699000,NaT,NaT,0 days 00:00:29.299000,0 days 00:00:39.355000,0 days 00:00:23.045000,None,240,267,281,313,5,0 days 07:06:42.833000,18,2025-02-28 13:51:43.028,0 days 00:01:31.699000
1344,17,0 days 00:01:32.124000,NaT,NaT,0 days 00:00:29.361000,0 days 00:00:39.631000,0 days 00:00:23.132000,None,240,266,281,310,4,0 days 06:39:03.521000,18,2025-02-28 13:24:03.716,0 days 00:01:32.124000
1346,19,0 days 00:01:32.484000,NaT,NaT,0 days 00:00:29.550000,0 days 00:00:39.693000,0 days 00:00:23.241000,None,240,267,281,311,4,0 days 06:42:37.571000,18,2025-02-28 13:27:37.766,0 days 00:01:32.484000
1338,11,0 days 00:01:33.185000,NaT,NaT,0 days 00:00:29.374000,0 days 00:00:39.704000,0 days 00:00:24.107000,None,240,266,281,311,3,0 days 06:19:10.225000,18,2025-02-28 13:04:10.420,0 days 00:01:33.185000
1351,24,0 days 00:01:32.412000,NaT,NaT,0 days 00:00:29.557000,0 days 00:00:39.754000,0 days 00:00:23.101000,None,240,267,280,313,5,0 days 07:10:18.881000,18,2025-02-28 13:55:19.076,0 days 00:01:32.412000
1341,14,0 days 00:01:32.694000,NaT,NaT,0 days 00:00:29.598000,0 days 00:00:39.974000,0 days 00:00:23.122000,None,240,266,280,311,3,0 days 06:23:24.678000,18,2025-02-28 13:08:24.873,0 days 00:01:32.694000
1333,6,0 days 00:01:33.227000,NaT,NaT,0 days 00:00:29.828000,0 days 00:00:40.018000,0 days 00:00:23.381000,None,236,268,279,312,2,0 days 06:04:09.503000,18,2025-02-28 12:49:09.698,0 days 00:01:33.227000
1358,31,0 days 00:01:36.213000,NaT,NaT,0 days 00:00:30.984000,0 days 00:00:41.271000,0 days 00:00:23.958000,None,233,259,271,283,6,0 days 07:30:29.354000,18,2025-02-28 14:15:29.549,0 days 00:01:36.213000
1357,30,0 days 00:01:36.324000,NaT,NaT,0 days 00:00:31.045000,0 days 00:00:41.334000,0 days 00:00:23.945000,None,None,257,271,285,6,0 days 07:28:54.254000,18,2025-02-28 14:13:54.449,0 days 00:01:36.324000
1359,32,0 days 00:01:36.483000,NaT,NaT,0 days 00:00:31.011000,0 days 00:00:41.422000,0 days 00:00:24.050000,None,None,254,274,284,6,0 days 07:32:05.678000,18,2025-02-28 14:17:05.873,0 days 00:01:36.483000


In [3]:
df_td = session.get_data("TimingData")
df_td6 = df_td[df_td["DriverNo"] == "18"].sort_values("timestamp",ascending=False)

In [4]:
df_td6

,SessionKey,timestamp,DriverNo,TimeDiffToFastest,TimeDiffToPositionAhead,Line,Position,ShowPosition,RacingNumber,Retired,...,Sectors_2_Segments_5_Status,Sectors_2_Segments_6_Status,Sectors_2_PreviousValue,Sectors_0_Value,Sectors_0_OverallFastest,Sectors_0_PersonalFastest,Sectors_0_PreviousValue,NumberOfPitStops,BestLapTime_Lap,_deleted
46416,9685,09:08:14.338,18,+2.154,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
44848,9685,08:55:50.032,18,NaN,+0.460,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
43901,9685,08:40:24.460,18,NaN,+0.238,13.0,13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
43365,9685,08:34:54.765,18,+2.133,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
41490,9685,08:20:30.885,18,+2.049,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20604,9685,04:39:53.911,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
20594,9685,04:39:53.077,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,[Lap]
20584,9685,04:39:48.988,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
20582,9685,04:39:47.490,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,26.030,NaN,NaN,NaN,31.012,NaN,NaN,False
